### Mail Gönderme İşlemi


Otomatik olarak mail göndermek için Python'da smtplib modülünü kullanabiliriz. Bu modül, SMTP (Simple Mail Transfer Protocol) sunucularına bağlanarak e-posta göndermemizi sağlar. Aşağıda, smtplib modülünü kullanarak e-posta gönderme işlemini adım adım göreceğiz.

1. E-posta göndermek için bir SMTP sunucusuna bağlanın.
2. E-posta başlığı, gövdesi ve alıcı adresi gibi e-posta bilgilerini belirtin.
3. E-posta gönderme işlemini gerçekleştirin.

Not: Outlook, Gmail veya Yahoo gibi e-posta sağlayıcılarının SMTP sunucu ayarlarını kullanarak e-posta gönderme işlemini gerçekleştirebilirsiniz. Örneğin, Outlook.com için SMTP sunucu ayarları şunlardır:

- Sunucu Adı: smtp.office365.com
- Port: 587
- Güvenlik: STARTTLS


In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import ssl

# Email configuration
email_sender = 'account'
email_password = 'passoword'
email_receiver = 'receiveraccount'
email_subject ='Veri Topluluğu VeriDay Katılım Sertifikaları'
email_body = """Sayın Katılımcı,

Veri Topluluğu tarafından düzenlenen VeriDay etkinliğine katılımınız için teşekkür ederiz. Sizlerin değerli katkılarıyla etkinliğimiz daha da anlamlı bir hale geldi.

Katılımınızı onurlandırmak adına size katılım sertifikasını sunuyoruz. Bu sertifika, VeriDay etkinliğine gösterdiğiniz ilgi ve katkılarınızın bir simgesi olarak size verilmiştir.

Katılım sertifikanızı ekte bulabilirsiniz. Ekteki belgede adınız ve etkinlik tarihleri yer almaktadır.

Yeniden katılımınız için teşekkür eder, başarılarınızın devamını dileriz.

İyi günler dileriz.

Saygılarımızla,

Veri Topluluğu
"""


file_path = 'certificatetemplate.jpg'  # Replace with the path to your file

# Create a multipart message
em = MIMEMultipart()
em['From'] = email_sender
em['To'] = email_receiver
em['Subject'] = email_subject

# Attach the email body
em.attach(MIMEText(email_body, 'plain'))

# Attach the file
with open(file_path, 'rb') as attachment:
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())

encoders.encode_base64(part)
part.add_header(
    'Content-Disposition',
    f'attachment; filename= {file_path}',
)
em.attach(part)

# Create a secure SSL context
context = ssl.create_default_context()

# Connect to the SMTP server using STARTTLS
with smtplib.SMTP('smtp.office365.com', 587) as server:
    server.starttls(context=context)
    server.login(email_sender, email_password)
    server.sendmail(email_sender, email_receiver, em.as_string())


### Sertifika Oluşturma İşlemi

Bu kısımda otomatik olarak sertifika oluşturma işlemini gerçekleştireceğiz.

Sertifika oluşturma işlemi için Pillow ve OpenCV kütüphanelerini kullanabiliriz. Pillow, Python için bir resim işleme kütüphanesidir ve metin çizme, resim birleştirme, resim döndürme gibi işlemleri gerçekleştirebilir. OpenCV ise görüntü işleme ve analizi için kullanılan bir kütüphanedir. Aşağıda, Pillow ve OpenCV kütüphanelerini kullanarak sertifika oluşturma işlemini adım adım göreceğiz.

1. Metni çizmek için Pillow kütüphanesini kullanın.
2. Metni OpenCV formatına dönüştürün.
3. Sertifika şablonunu yükleyin.
4. Metni sertifika üzerine yerleştirin.
5. Sertifikayı kaydedin.

Bu adımları takip ederek, sertifika oluşturma işlemini gerçekleştirebiliriz.

In [ ]:
import os
import cv2
from PIL import Image, ImageDraw, ImageFont
import numpy as np

list_of_names = []

def delete_old_data():
    for i in os.listdir("generated/"):
        os.remove("generated/{}".format(i))

def cleanup_data():
    with open('name-data.txt', encoding='utf-8') as f:
        for line in f:
            name = line.strip()
            if name.endswith(','):
                name = name[:-1]
            list_of_names.append(name)

def generate_certificates():
    for index, name in enumerate(list_of_names):
        # Pillow ile metni çiz
        font_path = "C:/Users/FurkanBulut/PycharmProjects/CertificateGenerator/TT Nooks Trial Light.otf"  # Özel fontunuzun yolu
        font_size = 80
        font = ImageFont.truetype(font_path, font_size)
        text_width, text_height = font.getsize(name)

        # Yeni bir görüntü oluştur
        text_image = Image.new('RGB', (text_width, text_height), color=(255, 255, 255))
        draw = ImageDraw.Draw(text_image)
        draw.text((0, 0), name, font=font, fill=(0, 0, 0))

        # Pillow görüntüsünü OpenCV formatına dönüştür
        opencv_text_image = np.array(text_image)
        opencv_text_image = opencv_text_image[:, :, ::-1].copy()  # RGB'den BGR'ye dönüşüm

        # Sertifika şablonunu yükle
        certificate_template_image = cv2.imread("certificatetemplate.jpg")

        # Metni sertifika üzerine yerleştir
        x_offset = int((certificate_template_image.shape[1] - text_width) / 2)
        y_offset = 400
        certificate_template_image[y_offset:y_offset+text_height, x_offset:x_offset+text_width] = opencv_text_image

        # Resmi kaydet
        cv2.imwrite(f"generated/{name}.jpg", certificate_template_image)
        print(f"İşleniyor: {index + 1} / {len(list_of_names)}")

def main():
    delete_old_data()
    cleanup_data()
    generate_certificates()

if __name__ == '__main__':
    main()


### Seritifika Oluşturma ve Mail Gönderme İşlemi

Bu kod otomatik olarak sertifika oluşturma ve e-posta gönderme işlemlerini gerçekleştirecektir. Bu işlem için aşağıdaki adımları takip edeceğiz:

1. E-posta göndermek için bir SMTP sunucusuna bağlanın.
2. E-posta başlığı, gövdesi ve alıcı adresi gibi e-posta bilgilerini belirtin.
3. E-posta gönderme işlemini gerçekleştirin.
4. Sertifika oluşturma işlemini gerçekleştirin.
5. Oluşturulan sertifikaları e-posta ile gönderin.
6. Excel dosyasından verileri okuyun ve sertifikaları oluşturun ve e-posta ile gönderin.

Bu işlemi gerçekleştirmek için aşağıdaki kodu kullanabilirsiniz.



In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import ssl
import os
import pandas as pd
import cv2
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# Email configuration
email_sender = 'outlookacıcynt'
email_password = 'passwordw'
email_subject = 'Veri Topluluğu VeriDay Katılım Sertifikaları'
email_body = """Sayın Katılımcı,

Veri Topluluğu tarafından düzenlenen VeriDay etkinliğine katılımınız için teşekkür ederiz. Sizlerin değerli katkılarıyla etkinliğimiz daha da anlamlı bir hale geldi.

Katılımınızı onurlandırmak adına size katılım sertifikasını sunuyoruz. Bu sertifika, VeriDay etkinliğine gösterdiğiniz ilgi ve katkılarınızın bir simgesi olarak size verilmiştir.

Katılım sertifikanızı ekte bulabilirsiniz. Ekteki belgede adınız ve etkinlik tarihleri yer almaktadır.

Yeniden katılımınız için teşekkür eder, başarılarınızın devamını dileriz.

İyi günler dileriz.

Saygılarımızla,

Veri Topluluğu
"""

file_path = 'certificatetemplate.jpg'  # Replace with the path to your certificate template file
data_file = 'ct.xlsx'  # Excel file containing names and emails

# Create a secure SSL context
context = ssl.create_default_context()

def send_email(receiver_email, attachment_path):
    em = MIMEMultipart()
    em['From'] = email_sender
    em['To'] = receiver_email
    em['Subject'] = email_subject

    em.attach(MIMEText(email_body, 'plain'))

    with open(attachment_path, 'rb') as attachment:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment.read())

    encoders.encode_base64(part)
    part.add_header(
        'Content-Disposition',
        f'attachment; filename= {attachment_path}',
    )
    em.attach(part)

    with smtplib.SMTP('smtp.office365.com', 587) as server:
        server.starttls(context=context)
        server.login(email_sender, email_password)
        server.sendmail(email_sender, receiver_email, em.as_string())

def delete_old_data():
    for i in os.listdir("generated/"):
        os.remove("generated/{}".format(i))

def generate_certificate(name, email):
    # Certificate generation logic
    font_path = "/content/TT Nooks Trial Light.otf"  # Path to your custom font
    font_size = 80
    font = ImageFont.truetype(font_path, font_size)
    draw = ImageDraw.Draw(Image.new('RGB', (1, 1)))
    text_size = draw.textsize(name, font=font)
    text_width, text_height = text_size
    text_image = Image.new('RGB', (text_width, text_height), color=(255, 255, 255))
    draw = ImageDraw.Draw(text_image)
    draw.text((0, 0), name, font=font, fill=(0, 0, 0))
    opencv_text_image = np.array(text_image)
    opencv_text_image = opencv_text_image[:, :, ::-1].copy()
    certificate_template_image = cv2.imread(file_path)
    x_offset = int((certificate_template_image.shape[1] - text_width) / 2)
    y_offset = 400
    certificate_template_image[y_offset:y_offset+text_height, x_offset:x_offset+text_width] = opencv_text_image
    certificate_name = f"VeriDay.jpg"
    cv2.imwrite(certificate_name, certificate_template_image)
    return certificate_name

def main():
    delete_old_data()

    # Read data from Excel file
    df = pd.read_excel(data_file)
    for index, row in df.iterrows():
        name = row['Name']
        email = row['Email']
        certificate_path = generate_certificate(name, email)
        send_email(email, certificate_path)
        print(f"Email sent to {name} ({email})")

if __name__ == '__main__':
    main()
